## Updating Constraint Matrices

To implement Graph SLAM, a matrix and a vector (omega and xi, respectively) are introduced. The matrix is square and labelled with all the robot poses (xi) and all the landmarks (Li). Every time you make an observation, for example, as you move between two poses by some distance `dx` and can relate those two positions, you can represent this as a numerical relationship in these matrices.

Below you can see a matrix representation of omega and a vector representation of xi.

<img src='images/omega_xi.png' width=20% height=20% />


### Solving for x, L

To "solve" for all these poses and landmark positions, we can use linear algebra; all the positional values are in the vector `mu` which can be calculated as a product of the inverse of omega times xi.

---


## Constraint Updates

In the below code, we construct `omega` and `xi` constraint matrices, and update these according to landmark sensor measurements and motion.

#### Sensor Measurements

When you sense a distance, `dl`, between a pose and a landmark, l, update the constraint matrices as follows:
* Add `[[1, -1], [-1, 1]]` to omega at the indices for the intersection of `xt` and `l`
* Add `-dl` and `dl` to xi at the rows for `xt` and `l`

The values 2 instead of 1 indicate the "strength" of the measurement.

You'll see three new `dl`'s as new inputs to our function `Z0, Z1, Z2`, below.

#### Motion
When your robot moves by some amount `dx` update the constraint matrices as follows:
* Add `[[1, -1], [-1, 1]]` to omega at the indices for the intersection of `xt` and `xt+1`
* Add `-dx` and `dx` to xi at the rows for `xt` and `xt+1`

## QUIZ: Include three new sensor measurements for a single landmark, L.

In [1]:
import numpy as np

In [3]:
def mu_from_positions(initial_pos: int, move1: int, move2: int, Z0: int, Z1: int, Z2: int) -> np.ndarray:
       ## TODO: construct constraint matrices
       # ## and add each position/motion constraint to them
    
       # initialize constraint matrices with 0's
       # Now these are 4x4 because of 3 poses and a landmark
       omega: np.ndarray = np.zeros(shape=(4,4))
       xi: np.ndarray = np.zeros(shape=(4,1))
       
       # add initial pose constraint
       omega[0][0] = 1
       xi[0] = initial_pos
       
       # account for the first motion, dx = move1
       omega += np.array([[1., -1., 0., 0.],
                     [-1., 1., 0., 0.],
                     [0., 0., 0., 0.],
                     [0., 0., 0., 0.]])
       xi += np.array([[-move1],
              [move1],
              [0.],
              [0.]])
       
       # account for the second motion
       omega += np.array([[0., 0., 0., 0.],
                     [0., 1., -1., 0.],
                     [0., -1., 1., 0.],
                     [0., 0., 0., 0.]])
       xi += np.array([[0.],
              [-move2],
              [move2],
              [0.]])
       
       
       ## TODO: Include three new sensor measurements for the landmark, L
       
       # Your code here
       
       # Update for first measurement Z0
       omega += np.array(
              [[1., 0., 0., -1.],
              [0., 0., 0., 0.],
              [0., 0., 0., 0.],
              [-1., 0., 0., 1.]])
       xi += np.array(
              [[-Z0],
              [0.],
              [0.],
              [Z0]])
       # Update for second measurement Z1
       omega += np.array(
              [[0., 0., 0., 0.],
              [0., 1., 0., -1.],
              [0., 0., 0., 0.],
              [0., -1., 0., 1.]])
       xi += np.array(
              [[0.],
              [-Z1],
              [0.],
              [Z1]])

       # Update for third measurement Z2
       omega += np.array(
              [[0., 0., 0., 0.],
              [0., 0., 0., 0.],
              [0., 0., 1., -1.],
              [0., 0., -1., 1.]])
       xi += np.array(
              [[0.],
              [0.],
              [-Z2],
              [Z2]])
       
       # display final omega and xi
       print('Omega: \n', omega)
       print('\n')
       print('Xi: \n', xi)
       print('\n')
       
       ## TODO: calculate mu as the inverse of omega * xi
       ## recommended that you use: np.linalg.inv(np.matrix(omega)) to calculate the inverse
       omega_inv: np.ndarray = np.linalg.inv(np.matrix(omega))
       mu: np.ndarray = omega_inv @ xi
       return mu

In [5]:
# call function and print out `mu`
mu = mu_from_positions(initial_pos=-3, move1=5, move2=3, Z0=10, Z1=5, Z2=2)
print('Mu: \n', mu)

Omega: 
 [[ 3. -1.  0. -1.]
 [-1.  3. -1. -1.]
 [ 0. -1.  2. -1.]
 [-1. -1. -1.  3.]]


Xi: 
 [[-18.]
 [ -3.]
 [  1.]
 [ 17.]]


Mu: 
 [[-3.]
 [ 2.]
 [ 5.]
 [ 7.]]


The landmark is at the location 7, which is also evident from the initial location and the first measurement.

Next we ake another experiment. Now we Earlier we supplied the initial position, two moves and respective sensor measurements to the landmark. Now let's say, that the sensor measurement for the last position to the landmark has decreased by 1.

<img src="./images/landmark.png" alt="Change in landmark sensor measurement">

And now we want to see, what estimates for the poses and landmark we get.

In [6]:
# call function and print out `mu`
mu = mu_from_positions(initial_pos=-3, move1=5, move2=3, Z0=10, Z1=5, Z2=1)
print('Mu: \n', mu)

Omega: 
 [[ 3. -1.  0. -1.]
 [-1.  3. -1. -1.]
 [ 0. -1.  2. -1.]
 [-1. -1. -1.  3.]]


Xi: 
 [[-18.]
 [ -3.]
 [  2.]
 [ 16.]]


Mu: 
 [[-3.   ]
 [ 2.125]
 [ 5.5  ]
 [ 6.875]]


Now we see, that due to change in sensor measurement Graph SLAM provides new locations. It now says that the landmark has got a bit closer to the last position to the robot and the robot's first and second position have also shifted a bit to the right. But expectedly the initial position did not change and we did not expect it either.